# Cordex preprocessing

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import xclim
xclim.__version__

'0.25.0'

In [3]:
import os
import intake
import xarray as xr
import numpy as np
from tqdm.notebook import tqdm

os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

xr.set_options(keep_attrs=True)
print(np.__version__)
print(xr.__version__)

import intake_esm
print(intake_esm.__version__)

import cordex as cx
from cordex.preprocessing import preprocessing as preproc

1.20.1
0.20.1
2021.1.15


In [4]:
eur11 = cx.cordex_domain('EUR-11')

In [5]:
!echo $HDF5_USE_FILE_LOCKING

FALSE


In [6]:
from dask.distributed import Client, progress
client = Client()

/work/ch0636/g300046/conda_envs/cmip6-xclim/lib/python3.9/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39975 instead
  warnings.warn(


In [7]:
client

Client Scheduler: tcp://127.0.0.1:41027 Dashboard: /user/g300046/advanced//proxy/39975/status,Cluster Workers: 6 Cores: 24 Memory: 31.25 GiB


In [8]:
#client.get_versions(check=True)

## Data access

In [9]:
url = "/work/kd0956/Catalogs/mistral-cordex.json" 
cat = intake.open_esm_datastore(url)
cat

,unique
product_id,3
CORDEX_domain,34
institute_id,95
driving_model_id,18
experiment_id,5
member,6
model_id,65
rcm_version_id,34
frequency,7
variable_id,105


In [10]:
models = ['CLMcom-CCLM4-8-17',
     'CLMcom-BTU-CCLM4-8-17',
    'CLMcom-ETH-COSMO-crCLIM-v1-1',
     'CNRM-ALADIN53',
     'CNRM-ALADIN63',
     'DMI-HIRHAM5',
     'GERICS-REMO2015',
     'ICTP-RegCM4-6',  # regcm seems to make trouble when opening with to_dataset_dict..
     'IPSL-INERIS-WRF331F',
     'KNMI-RACMO22E',
    'MOHC-HadREM3-GA7-05',
     'MPI-CSC-REMO2009',
     'RMIB-UGent-ALARO-0',
     'SMHI-RCA4', 
     'UHOH-WRF361H']

In [11]:
models = ['UHOH-WRF361H']
#models = 'ICTP-RegCM4-6'

In [12]:
#institute_ids = ['CLMcom', 'CLMcom-BTU', 'CLMcom-ETH', 'CNRM', 'DMI', 'GERICS',
#       'IPSL-INERIS', 'KNMI', 'MOHC', 'MPI-CSC', 'RMIB-UGent', 'SMHI',
#       'UHOH', 'z_GERICS_KB']
# there are some missleading entries in the catalog, so we state all institute ids explicitly here
# to avoid wrong ones...
institute_ids =  ['CLMcom', 'CLMcom-BTU', 'CLMcom-ETH', 'CNRM', 'DMI', 'GERICS',
        'ICTP', 'IPSL-INERIS', 'KNMI', 'MOHC', 'MPI-CSC', 'RMIB-UGent',
        'SMHI', 'UHOH', 'z_GERICS_KB']

In [13]:
scens = ['rcp26', 'rcp45', 'rcp85']
expts = ['historical'] + scens
attrs = {'variable_id': ['tas'] , 'frequency': 'mon', 'CORDEX_domain': 'EUR-11', 
         'experiment_id': expts, 'rcm_version_id': 'v1', 'model_id': models,}
       #  'institute_id': institute_ids}

In [14]:
import pandas as pd
pd.set_option('display.max_rows', None)
selection = cat.search(**attrs)
selection.df.groupby(['model_id', 'institute_id', 'experiment_id', 'driving_model_id', 'member', 'frequency', 'rcm_version_id', 'version'])['variable_id'].unique().apply(list).to_frame()

variable_id
model_id     institute_id experiment_id driving_model_id member  frequency rcm_version_id version              
UHOH-WRF361H UHOH         historical    ICHEC-EC-EARTH   r12i1p1 mon       v1             v20181203       [tas]
                                        MIROC-MIROC5     r1i1p1  mon       v1             v20180717       [tas]
                                        MOHC-HadGEM2-ES  r1i1p1  mon       v1             v20170524       [tas]
                                        MPI-M-MPI-ESM-LR r1i1p1  mon       v1             v20170524       [tas]
                          rcp26         MPI-M-MPI-ESM-LR r1i1p1  mon       v1             v20170524       [tas]
                          rcp85         ICHEC-EC-EARTH   r12i1p1 mon       v1             v20181203       [tas]
                                        MIROC-MIROC5     r1i1p1  mon       v1             v20180717       [tas]
                                        MOHC-HadGEM2-ES  r1i1p1  mon       v1             v20170524       [tas]
                                        MPI-M-MPI-ESM-LR r1i1p1  mon       v1             v20170524       [tas]

In [15]:
selection.df.institute_id.unique()

array(['UHOH'], dtype=object)

In [16]:
#selection['EUR-11.MIROC-MIROC5.UHOH.UHOH-WRF361H.rcp85.mon'].df

In [17]:
preproc.rename_cordex

<function cordex.preprocessing.preprocessing.rename_cordex(ds, rename_dict=None)>

In [18]:
dset_dict = selection.to_dataset_dict(cdf_kwargs = {"use_cftime": True, "chunks": {}}, preprocess=preproc.rename_cordex)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'CORDEX_domain.driving_model_id.institute_id.model_id.experiment_id.frequency'


In [19]:
dset_dict_flatten = preproc.member_id_to_dset_id(dset_dict)

In [20]:
dset_dict.keys()

dict_keys(['EUR-11.MPI-M-MPI-ESM-LR.UHOH.UHOH-WRF361H.historical.mon', 'EUR-11.ICHEC-EC-EARTH.UHOH.UHOH-WRF361H.historical.mon', 'EUR-11.MOHC-HadGEM2-ES.UHOH.UHOH-WRF361H.historical.mon', 'EUR-11.MIROC-MIROC5.UHOH.UHOH-WRF361H.historical.mon', 'EUR-11.MOHC-HadGEM2-ES.UHOH.UHOH-WRF361H.rcp85.mon', 'EUR-11.MIROC-MIROC5.UHOH.UHOH-WRF361H.rcp85.mon', 'EUR-11.MPI-M-MPI-ESM-LR.UHOH.UHOH-WRF361H.rcp26.mon', 'EUR-11.MPI-M-MPI-ESM-LR.UHOH.UHOH-WRF361H.rcp85.mon', 'EUR-11.ICHEC-EC-EARTH.UHOH.UHOH-WRF361H.rcp85.mon'])

In [28]:
sort = preproc.sort_ds_dict_by_attr(dset_dict, 'model_id')

In [39]:
for ds_id, ds in dset_dict.items():
    print(ds_id)
    print(ds.tas.dims)

EUR-11.MPI-M-MPI-ESM-LR.UHOH.UHOH-WRF361H.historical.mon
('member', 'time', 'rlat', 'rlon')
EUR-11.ICHEC-EC-EARTH.UHOH.UHOH-WRF361H.historical.mon
('member', 'time', 'rlat', 'rlon')
EUR-11.MOHC-HadGEM2-ES.UHOH.UHOH-WRF361H.historical.mon
('member', 'time', 'rlat', 'rlon')
EUR-11.MIROC-MIROC5.UHOH.UHOH-WRF361H.historical.mon
('member', 'time', 'height', 'rlat', 'rlon')
EUR-11.MOHC-HadGEM2-ES.UHOH.UHOH-WRF361H.rcp85.mon
('member', 'time', 'rlat', 'rlon')
EUR-11.MIROC-MIROC5.UHOH.UHOH-WRF361H.rcp85.mon
('member', 'time', 'height', 'lat', 'lon')
EUR-11.MPI-M-MPI-ESM-LR.UHOH.UHOH-WRF361H.rcp26.mon
('member', 'time', 'rlat', 'rlon')
EUR-11.MPI-M-MPI-ESM-LR.UHOH.UHOH-WRF361H.rcp85.mon
('member', 'time', 'rlat', 'rlon')
EUR-11.ICHEC-EC-EARTH.UHOH.UHOH-WRF361H.rcp85.mon
('member', 'time', 'rlat', 'rlon')


In [21]:
dset_sorted = preproc.sort_ds_dict_by_attr(dset_dict_flatten, 'experiment_id')

In [22]:
dset_sorted['rcp45'].keys()

dict_keys([])

In [21]:
ds_list = []
for ds in dset_sorted['rcp85'].values():
    ds = preproc.replace_rlon_rlat(ds)
    ds = preproc.replace_lon_lat(ds)
    ds_list.append(ds)

In [22]:
from xclim.ensembles import create_ensemble

In [28]:
rcp85 = create_ensemble(ds_list, resample_freq='MS')

/work/ch0636/g300046/conda_envs/cmip6-xclim/lib/python3.9/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]
/work/ch0636/g300046/conda_envs/cmip6-xclim/lib/python3.9/site-packages/xarray/core/indexing.py:1369: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  retur

In [ ]:
for key, ds in dset_dict.items():
    print(key)
    print(list(ds.dims))
    ds = preproc.rename_cordex(ds)
    preproc.check_domain(ds)

In [92]:
ds = dset_dict['EUR-11.NCC-NorESM1-M.z_GERICS_KB.GERICS-REMO2015.historical.mon']
ds = preproc.rename_cordex(ds)
ds = preproc.promote_empty_dims(ds)
ds = preproc.replace_rlon_rlat(ds)
ds

<xarray.Dataset>
Dimensions:                     (time: 671, bnds: 2, rlat: 412, rlon: 424, vertices: 4, member: 1)
Coordinates:
  * time                        (time) object 1950-02-15 00:00:00 ... 2005-12...
  * rlat                        (rlat) float64 -23.38 -23.27 ... 21.73 21.84
  * rlon                        (rlon) float64 -28.38 -28.27 ... 18.05 18.16
    lat                         (rlat, rlon) float32 dask.array<chunksize=(412, 424), meta=np.ndarray>
    lon                         (rlat, rlon) float32 dask.array<chunksize=(412, 424), meta=np.ndarray>
  * member                      (member) <U6 'r1i1p1'
  * bnds                        (bnds) int64 0 1
  * vertices                    (vertices) int64 0 1 2 3
Data variables:
    time_bounds                 (time, bnds) object dask.array<chunksize=(11, 2), meta=np.ndarray>
    rotated_latitude_longitude  int32 ...
    lat_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    lon_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    height                      float64 ...
    tas                         (member, time, rlat, rlon) float32 dask.array<chunksize=(1, 11, 412, 424), meta=np.ndarray>
Attributes: (12/36)
    source:                         GERICS-REMO2015
    driving_experiment_name:        historical
    experiment_id:                  historical
    comment:                        CORDEX Europe RCM REMO EUR-11. Data_polic...
    physics_version:                1
    rcm_version_id:                 v1
    ...                             ...
    intake_esm_varname:             ['tas']
    Conventions:                    CF-1.4
    branch_time:                    0.0
    experiment:                     historical
    parent_experiment_rip:          N/A
    intake_esm_dataset_key:         EUR-11.NCC-NorESM1-M.z_GERICS_KB.GERICS-R...

In [20]:
ds = dset_dict['EUR-11.NCC-NorESM1-M.CNRM.CNRM-ALADIN63.historical.mon']
ds
preproc.get_grid_mapping(ds)

<xarray.DataArray 'Lambert_Conformal' ()>
array(b'', dtype='|S1')
Coordinates:
    height   float64 ...
Attributes:
    latitude_of_projection_origin:  49.5
    standard_parallel:              49.5
    longitude_of_central_meridian:  10.5
    grid_mapping_name:              lambert_conformal_conic

In [21]:
ds = dset_dict['EUR-11.MPI-M-MPI-ESM-LR.GERICS.GERICS-REMO2015.historical.mon']
ds
preproc.get_grid_mapping(ds)

<xarray.DataArray 'rotated_latitude_longitude' ()>
array(-2147483647, dtype=int32)
Coordinates:
    height   float64 ...
Attributes:
    grid_mapping_name:          rotated_latitude_longitude
    grid_north_pole_latitude:   39.25
    grid_north_pole_longitude:  -162.0
    north_pole_grid_longitude:  0.0

In [84]:
preproc.get_grid_mapping(ds)

<xarray.DataArray 'Lambert_Conformal' ()>
array(b'', dtype='|S1')
Attributes:
    latitude_of_projection_origin:  49.5
    standard_parallel:              49.5
    longitude_of_central_meridian:  10.5
    grid_mapping_name:              lambert_conformal_conic

In [54]:
preproc.remap_lambert_conformal(ds)

/work/ch0636/g300046/conda_envs/cmip6-processing/lib/python3.9/site-packages/xesmf/frontend.py:464: FutureWarning: ``output_sizes`` should be given in the ``dask_gufunc_kwargs`` parameter. It will be removed as direct parameter in a future version.
  dr_out = xr.apply_ufunc(


<xarray.Dataset>
Dimensions:            (x: 485, y: 485, time: 1140, member: 1, rlat: 412, rlon: 424, bnds: 2)
Coordinates:
  * x                  (x) float64 0.0 12.5 25.0 ... 6.038e+03 6.05e+03
  * y                  (y) float64 0.0 12.5 25.0 ... 6.038e+03 6.05e+03
    lon                (y, x) float64 dask.array<chunksize=(485, 485), meta=np.ndarray>
    lat                (y, x) float64 dask.array<chunksize=(485, 485), meta=np.ndarray>
  * time               (time) object 2006-01-16 12:00:00 ... 2100-12-16 12:00:00
  * member             (member) <U6 'r1i1p1'
Dimensions without coordinates: rlat, rlon, bnds
Data variables:
    tas                (member, time, rlat, rlon) float64 dask.array<chunksize=(1, 60, 412, 424), meta=np.ndarray>
    Lambert_Conformal  |S1 ...
    time_bounds        (time, bnds) object dask.array<chunksize=(60, 2), meta=np.ndarray>
    height             float64 ...
Attributes: (12/23)
    Conventions:                    CF-1.6
    driving_experiment:             CNRM-CERFACS-CNRM-CM5, rcp85, r1i1p1
    driving_model_id:               CNRM-CERFACS-CNRM-CM5
    CORDEX_domain:                  EUR-11
    experiment_id:                  rcp85
    project_id:                     CORDEX
    ...                             ...
    tracking_id:                    5375b1a3-a3e6-4f8d-8aa1-684210096be9\n4c5...
    resolution_flag:                11
    rcm_version_id:                 v1
    frequency:                      mon
    driving_model_ensemble_member:  r1i1p1
    intake_esm_dataset_key:         EUR-11.CNRM-CERFACS-CNRM-CM5.RMIB-UGent.R...

In [41]:
preproc.regridder is None

True

In [42]:
preproc._init_regridder(ds, eur11)

In [43]:
preproc.regridder

xESMF Regridder 
Regridding algorithm:       bilinear 
Weight filename:            bilinear_485x485_412x424.nc 
Reuse pre-computed weights? False 
Input grid shape:           (485, 485) 
Output grid shape:          (412, 424) 
Periodic in longitude?      False

In [25]:
preproc.cordex_dataset_id(ds)

'EUR-11.CNRM-CERFACS-CNRM-CM5.RMIB-UGent.RMIB-UGent-ALARO-0.rcp85.mon.r1i1p1'

In [27]:
ds = dset_dict['EUR-11.IPSL-IPSL-CM5A-LR.GERICS.GERICS-REMO2015.historical.mon']
ds

<xarray.Dataset>
Dimensions:                     (time: 695, bnds: 2, rlat: 412, rlon: 424, vertices: 4, member: 1)
Coordinates:
  * time                        (time) object 1948-02-15 12:00:00 ... 2005-12...
  * rlat                        (rlat) float64 -23.38 -23.27 ... 21.72 21.84
  * rlon                        (rlon) float64 -28.38 -28.27 ... 18.04 18.16
    lat                         (rlat, rlon) float32 dask.array<chunksize=(412, 424), meta=np.ndarray>
    lon                         (rlat, rlon) float32 dask.array<chunksize=(412, 424), meta=np.ndarray>
    height                      float64 ...
  * member                      (member) <U6 'r1i1p1'
Dimensions without coordinates: bnds, vertices
Data variables:
    time_bnds                   (time, bnds) object dask.array<chunksize=(35, 2), meta=np.ndarray>
    rotated_latitude_longitude  int32 ...
    lat_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    lon_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    tas                         (member, time, rlat, rlon) float32 dask.array<chunksize=(1, 35, 412, 424), meta=np.ndarray>
Attributes: (12/30)
    experiment:                     historical
    project_id:                     CORDEX
    driving_experiment:             IPSL-IPSL-CM5A-LR, historical, r1i1p1
    product:                        output
    institution:                    Helmholtz-Zentrum Geesthacht, Climate Ser...
    tracking_id:                    4012e04b-6a54-494f-82b0-73e51fa26e77\nf05...
    ...                             ...
    source:                         GERICS-REMO2015
    rcm_version_id:                 v1
    driving_model_id:               IPSL-IPSL-CM5A-LR
    CORDEX_domain:                  EUR-11
    title:                          GERICS-REMO2015 model output prepared for...
    intake_esm_dataset_key:         EUR-11.IPSL-IPSL-CM5A-LR.GERICS.GERICS-RE...

We move the member id from a coordinate to the dsets key so that all datasets have the same structure...

we concentrate on rotated pole grids for now. there are just a handful of lambert conformal projection models that we can deal later with.

In [22]:
from cordex.preprocessing import preprocessing as preproc

In [23]:
dset_dict['EUR-11.ICHEC-EC-EARTH.DMI.DMI-HIRHAM5.historical.mon']

KeyError: 'EUR-11.ICHEC-EC-EARTH.DMI.DMI-HIRHAM5.historical.mon'

Now, we concatenate scenarion data with historical data for easier comparisons:

Now, we have to align the different coordinats. The ensemble members might have slightly different rotated coordinates and also the projection to the global coordinates might be different. We use a reference dataset here, from wich we copy the coordinates to all datasets to make them comparable.

The idea to make the whole dataset id a coordinate, so that xarray will automatically do all stuff for all datasets...

In [21]:
# make the dataset id a coordinate for easier access
dims = {}
for rcp, dsets in dsets_sorted.items():
    dset_ids = list(dsets.keys())
    dim = xr.DataArray(dset_ids, dims='dset_id', name='dset_id',
                      coords={'dset_id': dset_ids})
    dims[rcp] = dim

In [85]:
preproc.dset_ids_to_coord(dset_dict)

<xarray.DataArray 'dset_id' (dset_id: 47)>
array(['EUR-11.NCC-NorESM1-M.z_GERICS_KB.GERICS-REMO2015.historical.mon',
       'EUR-11.NOAA-GFDL-GFDL-ESM2G.GERICS.GERICS-REMO2015.historical.mon',
       'EUR-11.CNRM-CERFACS-CNRM-CM5.GERICS.GERICS-REMO2015.historical.mon',
       'EUR-11.MIROC-MIROC5.CLMcom.CLMcom-CCLM4-8-17.rcp26.mon',
       'EUR-11.IPSL-IPSL-CM5A-LR.GERICS.GERICS-REMO2015.rcp26.mon',
       'EUR-11.CNRM-CERFACS-CNRM-CM5.GERICS.GERICS-REMO2015.rcp85.mon',
       'EUR-11.NOAA-GFDL-GFDL-ESM2G.GERICS.GERICS-REMO2015.rcp26.mon',
       'EUR-11.MOHC-HadGEM2-ES.GERICS.GERICS-REMO2015.historical.mon',
       'EUR-11.MOHC-HadGEM2-ES.CLMcom.CLMcom-CCLM4-8-17.historical.mon',
       'EUR-11.MPI-M-MPI-ESM-LR.CLMcom.CLMcom-CCLM4-8-17.historical.mon',
       'EUR-11.ICHEC-EC-EARTH.CLMcom.CLMcom-CCLM4-8-17.historical.mon',
       'EUR-11.MIROC-MIROC5.GERICS.GERICS-REMO2015.historical.mon',
       'EUR-11.MPI-M-MPI-ESM-LR.GERICS.GERICS-REMO2015.historical.mon',
       'EUR-11.ICHEC-EC-EARTH.GERICS.GERICS-REMO2015.rcp26.mon',
       'EUR-11.MOHC-HadGEM2-ES.CLMcom.CLMcom-CCLM4-8-17.rcp85.mon',
       'EUR-11.NCC-NorESM1-M.z_GERICS_KB.GERICS-REMO2015.rcp85.mon',
       'EUR-11.IPSL-IPSL-CM5A-MR.GERICS.GERICS-REMO2015.rcp85.mon',
       'EUR-11.ICHEC-EC-EARTH.CLMcom.CLMcom-CCLM4-8-17.rcp26.mon',
       'EUR-11.NCC-NorESM1-M.GERICS.GERICS-REMO2015.historical.mon',
       'EUR-11.CCCma-CanESM2.GERICS.GERICS-REMO2015.historical.mon',
...
       'EUR-11.CNRM-CERFACS-CNRM-CM5.CLMcom.CLMcom-CCLM4-8-17.rcp85.mon',
       'EUR-11.CNRM-CERFACS-CNRM-CM5.CLMcom.CLMcom-CCLM4-8-17.historical.mon',
       'EUR-11.CCCma-CanESM2.CLMcom.CLMcom-CCLM4-8-17.historical.mon',
       'EUR-11.NCC-NorESM1-M.GERICS.GERICS-REMO2015.rcp85.mon',
       'EUR-11.MOHC-HadGEM2-ES.CLMcom.CLMcom-CCLM4-8-17.rcp45.mon',
       'EUR-11.MIROC-MIROC5.GERICS.GERICS-REMO2015.rcp85.mon',
       'EUR-11.CCCma-CanESM2.GERICS.GERICS-REMO2015.rcp85.mon',
       'EUR-11.MPI-M-MPI-ESM-LR.GERICS.GERICS-REMO2015.rcp85.mon',
       'EUR-11.IPSL-IPSL-CM5A-MR.GERICS.GERICS-REMO2015.historical.mon',
       'EUR-11.ICHEC-EC-EARTH.CLMcom.CLMcom-CCLM4-8-17.rcp85.mon',
       'EUR-11.ICHEC-EC-EARTH.GERICS.GERICS-REMO2015.rcp85.mon',
       'EUR-11.CCCma-CanESM2.CLMcom.CLMcom-CCLM4-8-17.rcp85.mon',
       'EUR-11.NCC-NorESM1-M.GERICS.GERICS-REMO2015.rcp45.mon',
       'EUR-11.MOHC-HadGEM2-ES.GERICS.GERICS-REMO2015.rcp85.mon',
       'EUR-11.MIROC-MIROC5.CLMcom.CLMcom-CCLM4-8-17.historical.mon',
       'EUR-11.MPI-M-MPI-ESM-LR.CLMcom.CLMcom-CCLM4-8-17.rcp85.mon',
       'EUR-11.NCC-NorESM1-M.GERICS.GERICS-REMO2015.rcp26.mon',
       'EUR-11.MOHC-HadGEM2-ES.GERICS.GERICS-REMO2015.rcp26.mon',
       'EUR-11.ICHEC-EC-EARTH.GERICS.GERICS-REMO2015.rcp45.mon'],
      dtype='<U68')
Coordinates:
  * dset_id  (dset_id) <U68 'EUR-11.NCC-NorESM1-M.z_GERICS_KB.GERICS-REMO2015...

we create three big datasets for each senario one:

In [120]:
from cordex import cordex_domain


def create_test_ds(name, pol_name='rotated_latitude_longitude'):
    domain = cordex_domain(name, mapping_name=pol_name, dummy=True, add_vertices=True)
    domain.dummy
    return domain
    


In [158]:
dm = create_test_ds('EUR-11', 'rotated_pole')
#dm = dm.drop(('lon', 'lat'))
#dm.rename({'rlon': 'lon', 'rlat': 'lat'})
dm['dummy'] = xr.DataArray(dm.dummy.values, dims=('lat', 'lon'), attrs=dm.dummy.attrs)
dm

<xarray.Dataset>
Dimensions:       (lat: 412, lon: 424, rlat: 412, rlon: 424, vertices: 4)
Coordinates:
  * rlon          (rlon) float64 -28.38 -28.27 -28.16 ... 17.94 18.05 18.16
  * rlat          (rlat) float64 -23.38 -23.27 -23.16 ... 21.62 21.73 21.84
    lon           (rlat, rlon) float64 -10.06 -9.964 -9.864 ... 64.76 64.96
    lat           (rlat, rlon) float64 21.99 22.03 22.07 ... 66.81 66.75 66.69
Dimensions without coordinates: vertices
Data variables:
    rotated_pole  int32 0
    lat_vertices  (rlat, rlon, vertices) float64 21.92 21.96 ... 66.7 66.76
    lon_vertices  (rlat, rlon, vertices) float64 -10.09 -9.99 ... 65.15 64.95
    dummy         (lat, lon) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [159]:
preproc.rename_cordex(dm)

<xarray.Dataset>
Dimensions:                     (rlat: 412, rlon: 424, vertices: 4)
Coordinates:
  * rlat                        (rlat) float64 -23.38 -23.27 ... 21.73 21.84
  * rlon                        (rlon) float64 -28.38 -28.27 ... 18.05 18.16
    lon                         (rlat, rlon) float64 -10.06 -9.964 ... 64.96
    lat                         (rlat, rlon) float64 21.99 22.03 ... 66.75 66.69
Dimensions without coordinates: vertices
Data variables:
    rotated_latitude_longitude  int32 0
    lat_vertices                (rlat, rlon, vertices) float64 21.92 ... 66.76
    lon_vertices                (rlat, rlon, vertices) float64 -10.09 ... 64.95
    dummy                       (rlat, rlon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0

In [123]:
preproc.rename_cordex(dm).equals(create_test_ds('EUR-11'))

True

In [154]:
dm = create_test_ds('EUR-11', 'rotated_pole')
#dm = dm.rename({'lon_vertices': 'longitude_vertices', 'lat_vertices': 'latitude_vertices'})
dm

<xarray.Dataset>
Dimensions:       (rlat: 412, rlon: 424, vertices: 4)
Coordinates:
  * rlon          (rlon) float64 -28.38 -28.27 -28.16 ... 17.94 18.05 18.16
  * rlat          (rlat) float64 -23.38 -23.27 -23.16 ... 21.62 21.73 21.84
    lon           (rlat, rlon) float64 -10.06 -9.964 -9.864 ... 64.76 64.96
    lat           (rlat, rlon) float64 21.99 22.03 22.07 ... 66.81 66.75 66.69
Dimensions without coordinates: vertices
Data variables:
    rotated_pole  int32 0
    lat_vertices  (rlat, rlon, vertices) float64 21.92 21.96 ... 66.7 66.76
    lon_vertices  (rlat, rlon, vertices) float64 -10.09 -9.99 ... 65.15 64.95
    dummy         (rlat, rlon) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

In [156]:
dm.drop_vars(('rlon', 'rlat'))

<xarray.Dataset>
Dimensions:       (rlat: 412, rlon: 424, vertices: 4)
Coordinates:
    lon           (rlat, rlon) float64 -10.06 -9.964 -9.864 ... 64.76 64.96
    lat           (rlat, rlon) float64 21.99 22.03 22.07 ... 66.81 66.75 66.69
Dimensions without coordinates: rlat, rlon, vertices
Data variables:
    rotated_pole  int32 0
    lat_vertices  (rlat, rlon, vertices) float64 21.92 21.96 ... 66.7 66.76
    lon_vertices  (rlat, rlon, vertices) float64 -10.09 -9.99 ... 65.15 64.95
    dummy         (rlat, rlon) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0

In [137]:
preproc.rename_cordex(dm)

<xarray.Dataset>
Dimensions:                     (rlat: 412, rlon: 424, vertices: 4)
Coordinates:
  * rlon                        (rlon) float64 -28.38 -28.27 ... 18.05 18.16
  * rlat                        (rlat) float64 -23.38 -23.27 ... 21.73 21.84
    lon                         (rlat, rlon) float64 -10.06 -9.964 ... 64.96
    lat                         (rlat, rlon) float64 21.99 22.03 ... 66.75 66.69
Dimensions without coordinates: vertices
Data variables:
    rotated_latitude_longitude  int32 0
    lat_vertices                (rlat, rlon, vertices) float64 21.92 ... 66.76
    lon_vertices                (rlat, rlon, vertices) float64 -10.09 ... 64.95
    dummy                       (rlat, rlon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0

In [139]:
preproc.rename_cordex(ds)

<xarray.Dataset>
Dimensions:                     (bnds: 2, height: 1, member: 1, rlat: 412, rlon: 424, time: 1140, vertices: 4)
Coordinates:
  * rlat                        (rlat) float64 -23.38 -23.26 ... 21.73 21.83
  * rlon                        (rlon) float64 -28.38 -28.26 ... 18.05 18.16
  * height                      (height) float64 2.0
  * time                        (time) object 2006-01-16 12:00:00 ... 2100-12...
  * member                      (member) <U6 'r1i1p1'
    lat                         (rlat, rlon) float32 dask.array<chunksize=(412, 424), meta=np.ndarray>
    lon                         (rlat, rlon) float32 dask.array<chunksize=(412, 424), meta=np.ndarray>
Dimensions without coordinates: bnds, vertices
Data variables:
    rotated_latitude_longitude  int32 ...
    tas                         (member, time, height, rlat, rlon) float32 dask.array<chunksize=(1, 60, 1, 412, 424), meta=np.ndarray>
    time_bounds                 (time, bnds) object dask.array<chunksize=(60, 2), meta=np.ndarray>
    lat_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    lon_vertices                (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
Attributes: (12/25)
    experiment:                     rcp85
    experiment_id:                  rcp85
    institution:                    Institute of Physics an Meteorology Unive...
    driving_model_id:               MIROC-MIROC5
    driving_experiment_name:        rcp85
    project_id:                     CORDEX
    ...                             ...
    product:                        output
    institute_id:                   UHOH
    institute_run_id:               hoh
    c3s_disclaimer:                 This data has been produced in the contex...
    Conventions:                    CF-1.4
    intake_esm_dataset_key:         EUR-11.MIROC-MIROC5.UHOH.UHOH-WRF361H.rcp...

In [99]:
xr.Dataset(dict)

<xarray.Dataset>
Dimensions:       (bnds: 2, height: 1, lat: 412, lon: 424, member: 1, rlat: 412, rlon: 424, time: 1140, vertices: 4)
Coordinates:
  * height        (height) float64 2.0
  * time          (time) object 2006-01-16 12:00:00 ... 2100-12-16 12:00:00
  * member        (member) <U6 'r1i1p1'
    lat           (rlat, rlon) float32 dask.array<chunksize=(412, 424), meta=np.ndarray>
    lon           (rlat, rlon) float32 dask.array<chunksize=(412, 424), meta=np.ndarray>
  * rlat          (rlat) float64 -23.38 -23.26 -23.16 ... 21.61 21.73 21.83
  * rlon          (rlon) float64 -28.38 -28.26 -28.16 ... 17.93 18.05 18.16
Dimensions without coordinates: bnds, vertices
Data variables:
    rotated_pole  int32 ...
    tas           (member, time, height, lat, lon) float32 dask.array<chunksize=(1, 60, 1, 412, 424), meta=np.ndarray>
    time_bnds     (time, bnds) object dask.array<chunksize=(60, 2), meta=np.ndarray>
    lat_vertices  (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>
    lon_vertices  (rlat, rlon, vertices) float32 dask.array<chunksize=(412, 424, 4), meta=np.ndarray>

In [105]:
preproc.rename_cordex(dm)

<xarray.Dataset>
Dimensions:                     (rlat: 412, rlon: 424, vertices: 4)
Coordinates:
  * rlat                        (rlat) float64 -23.38 -23.27 ... 21.73 21.84
  * rlon                        (rlon) float64 -28.38 -28.27 ... 18.05 18.16
    lon                         (rlat, rlon) float64 -10.06 -9.964 ... 64.96
    lat                         (rlat, rlon) float64 21.99 22.03 ... 66.75 66.69
Dimensions without coordinates: vertices
Data variables:
    rotated_latitude_longitude  int32 0
    lat_vertices                (rlat, rlon, vertices) float64 21.92 ... 66.76
    lon_vertices                (rlat, rlon, vertices) float64 -10.09 ... 64.95
    dummy                       (rlat, rlon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0

In [63]:
dm.dummy.reset_coords().assign_coords({'lat': dm.rlat.values, 'lon': dm.rlon.values})

<xarray.Dataset>
Dimensions:  (lat: 412, lon: 424, rlat: 412, rlon: 424)
Coordinates:
  * rlon     (rlon) float64 -28.38 -28.27 -28.16 -28.05 ... 17.94 18.05 18.16
  * rlat     (rlat) float64 -23.38 -23.27 -23.16 -23.05 ... 21.62 21.73 21.84
  * lon      (lon) float64 -28.38 -28.27 -28.16 -28.05 ... 17.94 18.05 18.16
  * lat      (lat) float64 -23.38 -23.27 -23.16 -23.05 ... 21.62 21.73 21.84
Data variables:
    dummy    (rlat, rlon) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0

In [66]:
dm.dummy.assign_coords({'rlat': dm.rlat.values, 'rlon': dm.rlon.values})

<xarray.DataArray 'dummy' (rlat: 412, rlon: 424)>
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])
Coordinates:
  * rlon     (rlon) float64 -28.38 -28.27 -28.16 -28.05 ... 17.94 18.05 18.16
  * rlat     (rlat) float64 -23.38 -23.27 -23.16 -23.05 ... 21.62 21.73 21.84
    lon      (rlat, rlon) float64 -10.06 -9.964 -9.864 ... 64.55 64.76 64.96
    lat      (rlat, rlon) float64 21.99 22.03 22.07 22.11 ... 66.81 66.75 66.69
Attributes:
    grid_mapping:  rotated_pole
    coordinates:   lon lat